In [1]:
#%pip install -Uq accelerate multiprocess

In [2]:
#%pip install --q langchain langchain_community openai faiss-cpu pandas tiktoken sentence-transformers

In [6]:
import dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [7]:
dotenv.load_dotenv()

# Initialize Sentence Transformer embeddings
model_name = "all-MiniLM-L6-v2"  # You can choose a different model if you prefer
embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [10]:
vectorstore = FAISS.load_local("../faiss_index", embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [11]:
query = "I'm an intermediate programmer with Python experience, looking to dive deep into machine learning for financial applications."
retrieved_docs = retriever.get_relevant_documents(query)

# Print the retrieved documents
for i, doc in enumerate(retrieved_docs, start=1):
    print(f"Document {i}: {doc}")

Document 1: page_content='Title: ACCA: Machine learning with Python for finance professionals\n    Subject: Computer Science\n    Description: A machine learning course focused on delivering practical Python skills for finance professionals looking to maximise their use of these time-saving tools within their organisation.\n    Level: Intermediate\n    Institution: ACCA\n    About: Welcome to Machine learning with Python for finance professionals, provided by ACCA (Association of Chartered Certified Accountants), the global body for professional accountants. This course is part of the FinTech for finance and business leaders professional certificate program. This course will provide a view of what lies under the surface of a machine learning output, help to better interrogate a model, and partner with data scientists and others in an organisation to drive adoption and use of machine learning. Digital finance knowledge and skills are essential components of the technology transformation

In [12]:
# Define a prompt template for course recommendations
prompt_template = """
As an AI course recommendation expert, provide personalized, high-quality suggestions based on the user's interests, goals, and background.

Chat History: {chat_history}
User Query: {question}
Relevant Courses: {context}

Response Guidelines:
1. Tone: Warm, professional, and approachable.
2. Analysis: Consider user's query, history, and educational needs.
3. Recommendations: For each course, include:
   - Title and institution
   - Brief overview
   - Skills to be gained
   - Key topics
   - Level, duration, language
   - Ratings (if available)
   - Course URL (if available)
4. Personalization: Explain how courses align with user's interests and needs.

Note: All recommendations will be drawn strictly from the provided course context.

Recommendation:
"""

In [13]:
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["chat_history", "question", "context"]
)

# Initialize the language model
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4o-mini")

/usr/local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [14]:
# Set up conversation memory with summarization
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=1000, memory_key="chat_history", return_messages=True)

In [15]:
# Create the conversational retrieval chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": PROMPT}
)

In [16]:
# Function to get course recommendations
def get_course_recommendations(user_query: str):
    result = qa_chain({"question": user_query})
    return result["answer"]

In [17]:
user_query1 = "I'm an intermediate programmer with Python experience, looking to dive deep into machine learning for financial applications."
result1 = get_course_recommendations(user_query1)
print("Recommendation:", result1)


/usr/local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Recommendation: Based on your background as an intermediate programmer with Python experience and your interest in diving deep into machine learning for financial applications, I have identified two relevant courses that align perfectly with your goals:

### 1. ACCA: Machine Learning with Python for Finance Professionals
- **Institution**: ACCA (Association of Chartered Certified Accountants)
- **Overview**: This course is specifically designed for finance professionals who want to enhance their understanding of machine learning using Python. It offers practical skills that can be directly applied in financial contexts, making it a great fit for your aspirations.
- **Skills to be Gained**:
  - Proficiency in Python for data analysis and machine learning.
  - Ability to automate Excel workflows using Python.
  - Understanding of how to interrogate machine learning models and work with data scientists.
- **Key Topics**:
  - Introduction to Python, data types, and control flows.
  - Using

In [18]:
user_query2 = "That sounds great! But I'm particularly interested in time series analysis and predictive modeling for stock markets. Any courses focusing on that?"
recommendation2 = get_course_recommendations(user_query2)
print("\nRecommendation 2:", recommendation2)


Recommendation 2: Based on your interest in time series analysis and predictive modeling for stock markets, I have identified several relevant courses that will help you deepen your understanding of these topics and apply them effectively in financial contexts. Here are my top recommendations:

### 1. Practical Time Series Analysis
- **Institution**: Coursera
- **Overview**: This course provides a hands-on approach to time series analysis, focusing on practical applications and techniques that can be used in various fields, including finance.
- **Skills to be Gained**:
  - Proficiency in time series forecasting.
  - Understanding of different time series models.
- **Key Topics**:
  - Time series data and its characteristics.
  - Forecasting methods and evaluation metrics.
  - Practical applications of time series analysis.
- **Level**: Intermediate
- **Duration**: Not specified
- **Language**: English
- **Ratings**: 4.6
- **Course URL**: [View Course](https://www.coursera.org/learn/pr